In [9]:
import yaml

class Monster(yaml.YAMLObject):
  yaml_tag = u'!Monster'
  def __init__(self, name, hp, ac, attacks):
    self.name = name
    self.hp = hp
    self.ac = ac
    self.attacks = attacks
  def __repr__(self): # 特殊方法、返回对象字符串表示
    return "%s(name=%r, hp=%r, ac=%r, attacks=%r)" % (
       self.__class__.__name__, self.name, self.hp, self.ac,      
       self.attacks)
obj = yaml.load("""
--- !Monster
name: Cave spider
hp: [2,6]    # 2d6
ac: 16
attacks: [BITE, HURT]
""",  Loader=yaml.FullLoader)
print(obj)


print(yaml.dump(Monster(name='Cave lizard', hp=[3,6], ac=16, attacks=['BITE','HURT'])))

Monster(name='Cave spider', hp=[2, 6], ac=16, attacks=['BITE', 'HURT'])
!Monster
ac: 16
attacks:
- BITE
- HURT
hp:
- 3
- 6
name: Cave lizard



## 所有的 Python 的用户定义类，都是 type 这个类的实例

In [12]:
class MyClass:
  pass

instance = MyClass()

print(type(instance))
print(type(MyClass))

<class '__main__.MyClass'>
<class 'type'>


## 用户自定义类，只不过是type类的__call__运算符重载

In [13]:
class MyClass:
  data = 1
  
instance = MyClass()
print(MyClass, instance, instance.data)

MyClass = type('MyClass', (), {'data': 1})
instance = MyClass()
print(MyClass, instance, instance.data)

<class '__main__.MyClass'> <__main__.MyClass object at 0x1069fd010> 1
<class '__main__.MyClass'> <__main__.MyClass object at 0x1063dbe00> 1


## metaclass 是 type 的子类，通过替换 type 的__call__运算符重载机制，“超越变形”正常的类。

In [16]:
class = type(classname, superclasses, attributedict) 
# 变为了
class = MyMeta(classname, superclasses, attributedict)


SyntaxError: invalid syntax (570394233.py, line 1)

# 自定义metaclass

In [20]:
class MyMeta(type):
    def __call__(cls, *args, **kwargs):
        print("元类的__call__方法被调用")
        return super().__call__(*args, **kwargs)

class MyClass(metaclass=MyMeta):
    def __new__(cls, *args, **kwargs):
        print("类的__new__方法被调用")
        return super().__new__(cls)

    def __init__(self, name):
        print("类的__init__方法被调用")
        self.name = name

obj = MyClass("张三")

元类的__call__方法被调用
类的__new__方法被调用
类的__init__方法被调用


In [27]:
class MyMeta(type):
    def __new__(cls, name, bases, attrs): # type实例创建前
        # 可以在这里对类的属性进行修改或添加
        new_attrs = {}
        for attr_name, attr_value in attrs.items():
            if not attr_name.startswith('__'):
                # 将所有非特殊属性名前加上'my_'前缀
                new_attrs[f'my_{attr_name}'] = attr_value
            else:
                new_attrs[attr_name] = attr_value
        # 创建类对象
        return super().__new__(cls, name, bases, new_attrs)

    def __init__(cls, name, bases, attrs): # type实例创建后
        super().__init__(name, bases, attrs)
        # 可以在这里进行一些类的初始化操作，比如打印类的信息
        print(f'创建了类：{name}')

class MyClass(metaclass=MyMeta):
    class_attr = "这是一个类属性"
    def __init__(self, score, clas):
        self.score = score
        self.clas = clas
    def original_method(self):
        print('这是原始方法')

# 创建MyClass的实例
my_obj = MyClass(100, 30)
# 调用修改后的方法
my_obj.my_original_method()  
print(my_obj.my_class_attr)

创建了类：MyClass
这是原始方法
这是一个类属性
